# Part 2 - Pigment Model V2

Trying a different modelling approach

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import statsmodels.api as sm
import sys
import os


# add SRC to path to allow imports
sys.path.append(os.path.join("..", "src"))

from make_dataset import make_dataset_from_raw

In [ ]:
path_calibr = os.path.join("..", "data", "raw", "calibration.csv")
path_sample = os.path.join("..", "data", "raw", "sample.csv")

df_calibr, info_calibr = make_dataset_from_raw(path_calibr)
df_sample, info_sample = make_dataset_from_raw(path_sample)

In [ ]:
cols = [
    "sample_sample",
    "dilution_sample",
    "wavelength_nm",
    "corrected_mean_absorption_sample"
]

df_combined = pd.concat(
    [df_calibr[cols], df_sample[cols]]
    ).copy()

df_combined.head()

In [ ]:
df_combined["sample_sample"].value_counts()

In [ ]:
colours = [
    "#1F77B4",
    "#FF7F0E",
    "#2CA02C",
    "#D62728",
    "#9467BD",
    "#8C564B",
    "#E377C2",
    "#7F7F7F",
    "#BCBD22",
    "#17BECF",
]

In [ ]:
fig = go.Figure()

# loop though each dilution and sample
samples = df_combined["sample_sample"].unique()
dilutions = df_combined["dilution_sample"].unique()
for i, sample in enumerate(samples):
    for j, dilution in enumerate(dilutions):
        df_plot = df_combined.query("dilution_sample == @dilution & sample_sample == @sample")
        # add main line
        fig.add_trace(
            go.Scatter(
                x=df_plot["wavelength_nm"],
                y=df_plot["corrected_mean_absorption_sample"],
                mode="lines",
                name=f"{sample}: {dilution}",
                line=dict(color=colours[i]),
            )
        )

fig.update_layout(
    title=f'Corrected Absorbance Spectra for callibration and sample data',
    xaxis_title="Wavelength (nm)",
    yaxis_title="Absorption (OD)",
    template="plotly_white",
)

fig.show()

# Modelling

Assume $l = 1$ and $A = \sum{A_i}$ for each chemical. 

$$ A = Ecl $$

We want to predict the concentration of the new sample given the absoprtion over each wavelength. 
Our current data only uses "dilution" and so fist we will convert to a concentration. 
Undiluted (dilution = 1) is assumed to be 50 mg/L. 

Given the linear relationship between measured absoption and concentration we will then fit a linear model.

We will also filter the wavelengths to between 470 nm and 570 nm as this section of the sample better matches the callibration data.

In [ ]:
df_combined_peak = df_combined.copy()

df_combined_peak["wavelength_nm"] = df_combined_peak["wavelength_nm"].astype(int)
df_combined_peak = df_combined_peak.query("wavelength_nm > 470 & wavelength_nm < 570")

initial_conc = 50
df_combined_peak["concentration_mg_l"] = initial_conc / df_combined_peak["dilution_sample"]

df_combined_peak.head()

Currently sample X1 has dilution of 1, but we don't know the concentration. The above filled this is as 50 mg/L but we will predict this later. 

In [ ]:
# Split data into S1 sample for fitting and X1 for prediction later

df_model_s1 = df_combined_peak[df_combined_peak["sample_sample"] == "S1"]
df_model_x1 = df_combined_peak[df_combined_peak["sample_sample"] == "X1"]


### Absorption only model

In [ ]:
cols_target = "concentration_mg_l"

cols_feature = [
    "corrected_mean_absorption_sample"
]

model = sm.OLS(
    df_model_s1[cols_target], 
    df_model_s1[cols_feature]
)
result = model.fit()
result.summary()

In [ ]:
# 44.24 mg/L was the peak-only model
conc = result.predict(df_model_x1[cols_feature]).mean()

conc

The concentration should be between 25 and 50 mg/l bases on data between the peak, or over 50 mg/l away from the peak.

This value is lower than the previous pigment notebook model, but is otherwise consistent with where we would expect it to be.

As there is no wavelngth dependence the absorption away from the peak absorbance is lower and so drags the average down, thus decreasing the predicted value. This is likely not a good model. 

In [ ]:
model_results = pd.DataFrame({
    "Model":["Peak Only", "Absorption only"], 
    "Conc" : [44.24, round(conc, 2)],
    "R Squared" : [np.nan, round(result.rsquared, 2)],
    "BIC" : [np.nan, round(result.bic)]
    })
model_results

### Aborption and wavelength model

In [ ]:
cols_target = "concentration_mg_l"

cols_feature = [
    "corrected_mean_absorption_sample",
    "wavelength_nm"
]

model = sm.OLS(
    df_model_s1[cols_target], 
    df_model_s1[cols_feature]
)
result = model.fit()
result.summary()

In [ ]:
# 44.24 mg/L was the peak-only model
conc = result.predict(df_model_x1[cols_feature]).mean()
conc

Again, similar to the previous model. There is a clear wavelength dependence, and the linear coefficent is significant. But, a simple linear term is still not enough to capture the complexity of the relationship. 

In [ ]:
model_results_tmp = pd.DataFrame({
    "Model":["Absorption and wavelength"], 
    "Conc" : [round(conc, 2)],
    "R Squared" : [round(result.rsquared, 2)],
    "BIC" : [round(result.bic)]
    })
model_results = pd.concat([model_results, model_results_tmp]).drop_duplicates()
model_results

### Absorbance wavelength and intercept

In [ ]:

df_model = df_model_s1.copy()

df_model["intercept"] = 1


cols_feature = [
    "intercept",
    "corrected_mean_absorption_sample",
    "wavelength_nm"
]
cols_target = "concentration_mg_l"

model = sm.OLS(
    df_model[cols_target], 
    df_model[cols_feature]
)
result = model.fit()
result.summary()


In [ ]:
df_predict = df_model_x1.copy()
df_predict["intercept"] = 1

conc = result.predict(df_predict[cols_feature]).mean()
conc

In [ ]:
model_results_tmp = pd.DataFrame({
    "Model":["Absorption, wavelength, intercept"], 
    "Conc" : [round(conc, 2)],
    "R Squared" : [round(result.rsquared, 2)],
    "BIC" : [round(result.bic)]
    })
model_results = pd.concat([model_results, model_results_tmp]).drop_duplicates()
model_results

### Absorbance with mulitple wavelength terms

In [ ]:
cols_target = "concentration_mg_l"



df_model = df_model_s1.copy()

df_model["intercept"] = 1
df_model["wavelength_nm_2"] = df_model["wavelength_nm"]**2


cols_feature = [
    "intercept",
    "corrected_mean_absorption_sample",
    "wavelength_nm",
    "wavelength_nm_2"
]

model = sm.OLS(
    df_model[cols_target], 
    df_model[cols_feature]
)
result = model.fit()
result.summary()

In [ ]:
df_predict = df_model_x1.copy()

df_predict["intercept"] = 1
df_predict["wavelength_nm_2"] = df_predict["wavelength_nm"]**2

conc = result.predict(df_predict[cols_feature]).mean()
conc

In [ ]:
model_results_tmp = pd.DataFrame({
    "Model":["Absorption multi wavelengths"], 
    "Conc" : [round(conc, 2)],
    "R Squared" : [round(result.rsquared, 2)],
    "BIC" : [round(result.bic)]
    })
model_results = pd.concat([model_results, model_results_tmp]).drop_duplicates()
model_results

As we have added more terms we have decreased our R-squared, but also decreased our BIC. Models with a lower BIC tend to be preferred.

This last model has a substantially lower BIC than the previous, and a slighlty higher R-squared suggesting this might be the best model so far? Interesting it also has the highest predicted concentration, closer to the peak only model which is how I used to work this out in the Lab myself. 

### Even more wavelength terms?

In [ ]:
cols_target = "concentration_mg_l"



df_model = df_model_s1.copy()

df_model["intercept"] = 1
df_model["wavelength_nm_2"] = df_model["wavelength_nm"]**2
df_model["wavelength_nm_3"] = df_model["wavelength_nm"]**3
df_model["wavelength_nm_4"] = df_model["wavelength_nm"]**4


cols_feature = [
    "intercept",
    "corrected_mean_absorption_sample",
    "wavelength_nm",
    "wavelength_nm_2",
    "wavelength_nm_3",
    "wavelength_nm_4",
]

model = sm.OLS(
    df_model[cols_target], 
    df_model[cols_feature]
)
result = model.fit()
result.summary()

In [ ]:
df_predict = df_model_x1.copy()

df_predict["intercept"] = 1
df_predict["wavelength_nm_2"] = df_predict["wavelength_nm"]**2
df_predict["wavelength_nm_3"] = df_predict["wavelength_nm"]**3
df_predict["wavelength_nm_4"] = df_predict["wavelength_nm"]**4


conc = result.predict(df_predict[cols_feature]).mean()
conc

In [ ]:
model_results_tmp = pd.DataFrame({
    "Model":["Absorption more multi wavelengths"], 
    "Conc" : [round(conc, 2)],
    "R Squared" : [round(result.rsquared, 2)],
    "BIC" : [round(result.bic)]
    })
model_results = pd.concat([model_results, model_results_tmp]).drop_duplicates()
model_results

Again, an even lower BIC and higher R Squared than the previous model, and a higher predicted wavelength. The wavelength to the forth power was not significant and so could probably be removed.  

### Each wavelength as a separate feature

Most values are colinear and so this doesn't work. Would need to do some more work to find components that would work. 

In [ ]:
# df_model = df_model_s1.copy()


# wavelengths = df_model["wavelength_nm"].unique()
# df_model = df_model.pivot(
#     index = ["concentration_mg_l"],
#     columns = "wavelength_nm",
#     values = "corrected_mean_absorption_sample"
# ).reset_index()

# df_model["intercept"] = 1


# cols_feature = [
#     "intercept",
#     *wavelengths
# ]
# cols_target = "concentration_mg_l"

# model = sm.OLS(
#     df_model[cols_target], 
#     df_model[cols_feature]
# )
# result = model.fit()
# result.summary()
